In [ ]:
import os, random, sys, time, shutil, re, csv, datetime, pickle, math 

import pandas as pd

from random import seed
from random import random

from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.webdriver import Options
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup

In [ ]:
chrome_options = Options()
#chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--log-level=3")  # fatal
chrome_options.add_argument("--start-maximized")

In [ ]:
browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

In [ ]:
browser.get("https://www.linkedin.com/uas/login")

In [ ]:
try:
    f = open("../cookies/cookies.pkl")
    f.close()

    print('[Status] Cookies found! Logging in')

    cookies = pickle.load(open("../cookies/cookies.pkl", "rb"))

    for cookie in cookies:
        browser.add_cookie(cookie)
except IOError:
    print('[Status] Cookies not found!')

    print('[Status] Reading username and password file')

    config = open('../config/config.txt')
    lines = config.readlines()
    config.close()
    username = lines[0]
    password = lines[1]

    print('[Status] Entering username and password in their respective fields')

    username_field = browser.find_element_by_id('username')
    username_field.send_keys(username)

    password_field = browser.find_element_by_id('password')
    password_field.send_keys(password)

    print('[Status] Submiting username and password...')

    try:
        password_field.submit()

        print('[Status] Saving cookies...')
        
        pickle.dump(browser.get_cookies(), open("../cookies/cookies.pkl","wb"))
        
        print('[Status] Successfully saved cookies!')
    except:
        pass

In [ ]:
#pickle.dump(browser.get_cookies(), open("../cookies/cookies.pkl","wb"))

In [ ]:
# seed the pseudorandom number generator
seed(1)

In [ ]:
def get_all_jobs(company, competitors, limit):
    
    #opening the csv to save the informations about the client
    try:
        df_info = pd.read_csv('client_info.csv')
    except:
        df_info = pd.DataFrame()
    
    #searching for the company's linkedin page
    print("searching for the company " + company)
    browser.get("https://www.linkedin.com/search/results/companies/?keywords=" + company)
    time.sleep(1 + random())
    
    #getting company id
    soup = BeautifulSoup(browser.page_source, features='html.parser')
    code = soup.find_all('code')
    
    company_id = ''
    
    try:
        for code_str in code:
            if('urn:li:fsd_company:' in code_str.text):
                company_id = re.findall(r"(?<=fsd_company:)\d+?(?=[,|\"])", code_str.text)[0]
                print(company_id)
                if(company_id != '8175'):
                    break
        if company_id == '':
            raise Exception("No company Id")
    except:
        print("company " + company + " not found\n")
        df_info = df_info.append({
            'client': company, 
            'found': False, 
            'client id': '',
            'competitors': competitors,
            'employees': '',
            'total jobs': '', 
            'collected jobs': ''
        }, ignore_index=True)
        df_info.to_csv("client_info.csv", index=False)
        return 
        
    #accessing the company's job offers
    browser.get("https://www.linkedin.com/jobs/search/?geoId=92000000&f_C=" + company_id )
    
    #getting the total pages of the job offers
    soup = BeautifulSoup(browser.page_source, features='html.parser')
    
    try:
        total_jobs = re.search(r"(.+\ )", soup.find("div", attrs={"class": "jobs-search-results-list__title-heading"}).find("small").text).group(1).strip()
    except:
        print("no jobs found for " + company + '\n')
        df_info = df_info.append({
            'client': company, 
            'found': True,
            'client id': str(company_id),
            'competitors': competitors, 
            'employees': '',
            'total jobs': '0', 
            'collected jobs': '0'
        }, ignore_index=True)
        df_info.to_csv("client_info.csv", index=False)
        return
    
    total_jobs = int(re.sub(r'[\.|,]', '', total_jobs))
    
    pages = math.ceil(int(total_jobs) / 25)
    print(str(total_jobs) + ' jobs and ' + str(pages) + ' pages found for ' + company + "\n")

    #getting the information from all the jobs
    jobs_computed = 0
    jobs = []
    
    for page in range(pages):
        
        #access a job page
        browser.get('https://www.linkedin.com/jobs/search/?geoId=92000000&f_CR=103644278&f_F=it%2Ceng&f_C=' + company_id + '&start=' + str(page * 25))
        
        try:
            element = WebDriverWait(browser, 3).until(
                EC.presence_of_element_located(
                    (By.CLASS_NAME, "artdeco-pagination__pages--number")
                )
            )

        except:
            print("The page has no pagination")
            break
        
        soup = BeautifulSoup(browser.page_source, features='html.parser')
        time.sleep(random())
        
        #get a list of jobs from the page
        li_list = soup.find_all("li",attrs={"data-occludable-entity-urn" :True}, recursive=True)

        for li in li_list:
            
            #get the id of the job
            job_number = re.findall(r"(?<=_jobPosting:)\d+?$", li.attrs['data-occludable-entity-urn'])[0]
                    
            jobs_computed += 1
            print("jobs computed: " + str(jobs_computed))
            
            #click on a job to show its details
            selector = 'li[data-occludable-entity-urn="urn:li:fs_normalized_jobPosting:' + str(job_number) + '"]'
            browser.find_element_by_css_selector(selector).click()
            time.sleep(random())

            try:
                element = WebDriverWait(browser, 2).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "jobs-apply-button--top-card"))
                )
            except:
                print("Failed to load Button")
            
            soup = BeautifulSoup(browser.page_source, features='html.parser')
            
            #start getting all the information
            try:
                job_description = soup.find(attrs={"class": "jobs-description-content__text"}).getText(separator=' ')
                job_description = re.sub(r'\ +', ' ', job_description.replace('\n', '')).strip()
            except:
                try:
                    time.sleep(1)
                    soup = BeautifulSoup(browser.page_source, features='html.parser')
                    job_description = soup.find(attrs={"class": "jobs-description-content__text"}).getText(separator=' ')
                    job_description = re.sub(r'\ +', ' ', job_description.replace('\n', '')).strip()
                except:
                    job_description = 'not found'
                
             
            try:
                applicants = soup.find(attrs={'class': 'jobs-details-job-summary'}).find(text=re.compile('\d\ applicant')).replace('\n', '').strip()
                applicants = re.sub(r'[a-zA-Z\ ]', '', applicants)
            except:
                try:
                    time.sleep(1)
                    soup = BeautifulSoup(browser.page_source, features='html.parser')
                    applicants = soup.find(attrs={'class': 'jobs-details-job-summary'}).find(text=re.compile('\d\ applicant')).replace('\n', '').strip()
                    applicants = re.sub(r'[a-zA-Z\ ]', '', applicants)
                except:
                    applicants = 'not found'
                    
            
            try:
                employees = soup.find(attrs={'class': 'jobs-details-job-summary'}).find(text=re.compile('[\+\d]\ employees')).replace('\n', '').strip()
                employees = re.sub(r'[a-zA-Z\ ]', '', employees)
            except:
                try:
                    time.sleep(1)
                    soup = BeautifulSoup(browser.page_source, features='html.parser')
                    employees = soup.find(attrs={'class': 'jobs-details-job-summary'}).find(text=re.compile('[\+\d]\ employees')).replace('\n', '').strip()
                    employees = re.sub(r'[a-zA-Z\ ]', '', employees)
                except:
                    employees = 'not found'
                    
            
            try:
                views = soup.find(attrs={'class': 'jobs-details-top-card__content-container'}).find(text=re.compile('\d\ view')).replace('\n', '').strip()
                views = re.sub(r'[a-zA-Z\ ]', '', views)
            except:
                try:
                    time.sleep(1)
                    soup = BeautifulSoup(browser.page_source, features='html.parser')
                    views = soup.find(attrs={'class': 'jobs-details-top-card__content-container'}).find(text=re.compile('\d\ view')).replace('\n', '').strip()
                    views = re.sub(r'[a-zA-Z\ ]', '', views)
                except:
                    views = 'not found'
            
            try:
                job_li = soup.find('li',{'data-occludable-entity-urn': 'urn:li:fs_normalized_jobPosting:' + job_number})
                job_date = job_li.find("time")["datetime"]
            except:
                try:
                    time.sleep(1)
                    soup = BeautifulSoup(browser.page_source, features='html.parser')
                    job_li = soup.find('li',{'data-occludable-entity-urn': 'urn:li:fs_normalized_jobPosting:' + job_number})
                    job_date = job_li.find("time")["datetime"]
                except:
                    job_date = 'not found'
            
            try:
                job_li = soup.find('li',{'data-occludable-entity-urn': 'urn:li:fs_normalized_jobPosting:' + job_number})
                location = job_li.find("li", {'class': 'job-card-container__metadata-item'}).get_text()
            except:
                try:
                    time.sleep(1)
                    soup = BeautifulSoup(browser.page_source, features='html.parser')
                    job_li = soup.find('li',{'data-occludable-entity-urn': 'urn:li:fs_normalized_jobPosting:' + job_number})
                    location = job_li.find("li", {'class': 'job-card-container__metadata-item'}).get_text()
                except:
                    location = 'not found'
            
            try:
                industries_functions = soup.find_all('ul', {"class": "jobs-box__list jobs-description-details__list"},recursive=True)

                title = soup.find('h2', {"class": "jobs-details-top-card__job-title"},recursive=True).text
                title = title.replace('\n', '').strip()

                company = soup.find('a', {"class": "jobs-details-top-card__company-url"},recursive=True).text
                company = company.replace('\n', '').strip()

                industry = []
                functions = []
                for li in industries_functions[0].find_all('li'):
                    industry.append(li.text.replace('\n', '').strip())
                for li in industries_functions[1].find_all('li'):
                    functions.append(li.text.replace('\n', '').strip())
            except:
                try:
                    time.sleep(1)
                    soup = BeautifulSoup(browser.page_source, features='html.parser')
                    industries_functions = soup.find_all('ul', {"class": "jobs-box__list jobs-description-details__list"},recursive=True)

                    title = soup.find('h2', {"class": "jobs-details-top-card__job-title"},recursive=True).text
                    title = title.replace('\n', '').strip()

                    company = soup.find('a', {"class": "jobs-details-top-card__company-url"},recursive=True).text
                    company = company.replace('\n', '').strip()

                    industry = []
                    functions = []
                    for li in industries_functions[0].find_all('li'):
                        industry.append(li.text.replace('\n', '').strip())
                    for li in industries_functions[1].find_all('li'):
                        functions.append(li.text.replace('\n', '').strip())
                except:
                    industry = ['not found']
                    functions = ['not found']
            
            #grouping job data found
            data={
                "title": title, 
                "company": company,
                "client id": company_id,
                "date": job_date,
                "applicants": applicants,
                "description": job_description,
                "views": views,
                "location": location, 
                "industry": ', '.join(industry), 
                "job functions": ', '.join(functions)
            }
            jobs.append(data)
            
            print("titulo: " + title)
            print("empresa: " + company)
            print("data " + job_date)
            print("applicants " + applicants)
            print("views " + views)
            print("location " + location)
            print("description " + job_description[:40] + '...')
            print("setores: " + ', '.join(industry))
            print("funcoes: " + ', '.join(functions))
            print("-----------------------------------------------------")
            
            if jobs_computed >= limit:
                break
        else:
            continue
        break
    
    #storing company data found
    df_info = df_info.append({
        'client': company, 
        'found': True, 
        'client id': company_id,
        'competitors': competitors, 
        'employees': employees,
        'total jobs': total_jobs, 
        'collected jobs': total_jobs if total_jobs < limit else limit
    }, ignore_index=True)
    df_info.to_csv("client_info.csv", index=False)
    
    #storing job data found
    try:
        df = pd.read_csv('jobs.csv')
        df_new = pd.DataFrame(jobs)
        df = pd.concat([df, df_new])#.drop_duplicates()
    except:
        df = pd.DataFrame(jobs)
    df.to_csv("jobs.csv", index=False)
    
    print("**************finished " + company + "\n")

In [ ]:
def get_usa_prospects_list():
    return pd.read_csv('usa_selection.csv')['prospects'].to_list()

In [ ]:
def get_jobs_companies_list():
    return pd.read_csv('jobs.csv')['company'].to_list()

In [ ]:
companies = get_usa_prospects_list()
#companies_jobs = get_jobs_companies_list()
#to_search = [c for c in companies if c not in companies_jobs]

start_index = 39

while start_index < len(companies):
    
    for i in range(start_index, len(companies)):
        try:
            print("Buscando:", companies[i])
            #get_all_jobs(companies[i], "teste_03_02", 200)
        except:
            print("Errou em:", i)
            pass
        start_index = start_index + 1

In [ ]:
df_jobs = pd.read_csv("jobs.csv")
df_jobs

In [ ]:
df_client_info = pd.read_csv("client_info.csv")
df_client_info

In [ ]:
df_client_info['found'].value_counts()

In [ ]:
df_client_info.loc[df_client_info['found'] == True]['total jobs'].value_counts()